In [88]:
import numpy as np
import random
import tensorflow as tf
from collections import deque
import gym
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [89]:
# env_name = 'CartPole-v1'
env = gym.make(env_name)


In [90]:
# env.reset()
# for step in range(100):
#     env.render()
#     env.step(env.action_space.sample())
#
# env.close()

In [91]:
num_of_obs = env.observation_space.shape[0]
num_of_act = env.action_space.n

In [92]:
model = Sequential()
model.add(Dense(16, activation = 'relu', input_shape=(1, num_of_obs)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(num_of_act, activation = 'linear'))



In [93]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 1, 16)             80        
                                                                 
 dense_16 (Dense)            (None, 1, 32)             544       
                                                                 
 dense_17 (Dense)            (None, 1, 2)              66        
                                                                 
Total params: 690 (2.70 KB)
Trainable params: 690 (2.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [94]:
target_model = clone_model(model)

In [95]:
EPOCHS = 1000
epsilon = 1.0
epsilon_reduce = 0.0001
learning_rate = 0.001
gamma = 0.95

In [96]:
def egas(model, epsilon, obs):
    if np.random.random() > epsilon:
        prediction = model.predict(obs)
        action = np.argmax(prediction)

    else:
        action = env.action_space.sample()

    return action

In [97]:
replay_buffer = deque(maxlen=20000)
update_target_model = 10

In [98]:
def replay(replay_buffer, batch_size, model, target_model):
    if len(replay_buffer) < batch_size:
        return
    samples = random.sample(replay_buffer, batch_size)
    target_batch = []
    zipped_samples = list(zip(*samples))
    states, actions, rewards, new_states, dones = zipped_samples
    targets = target_model.predict(np.array(states))
    q_value = model.predict(np.array(new_states))

    for i in range(batch_size):
        print(f"{q_value} : q vlaue")
        q_value = max(q_value[i][0])
        print(f"{q_value} : q max")
        target = targets[i].copy()

        if dones[i]:
            target[0][actions[i]] = rewards[i]

        else:
            target[0][actions[i]] = rewards[i]+q_value*gamma

        target_batch.append(target)

    model.fit(np.array(states), np.array(target_batch), epochs = 1)


In [99]:
def update_model_handler(update_target_model, model, target_model, epoch):
    # print(epoch)
    if epoch > 0 and update_target_model % epoch == 0:
        target_model.set_weights(model.get_weights())

In [100]:
model.compile(loss='mse', optimizer = (Adam(learning_rate=learning_rate)))

In [101]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

In [102]:
best = 0
for epoch in range(EPOCHS):
    obs = env.reset()
    # print(obs)
    obs = obs[0].reshape([1,4])
    done = False
    points = 0
    while not done:
        action = egas(model, epsilon, obs)
        # print(env.step(action))
        next_obs, reward, done, info, check = env.step(action)
        next_obs = next_obs.reshape([1,4])

        replay_buffer.append((obs, action, reward, next_obs, done))
        obs = next_obs
        points+=1
        replay(replay_buffer, 32, model, target_model)

    epsilon = epsilon - epsilon_reduce
    update_model_handler(update_target_model, model, target_model, epoch)
    if points > best:
        best = points

    if epoch%25 == 0:
        print(f"{epoch} : {points} points : {best} : best")


0 : 13 points : 13 : best
1/1 [==============================] - 0s 45ms/step
[[[-1.03079014e-01  3.25278550e-01]]

 [[-1.87837988e-01  5.69102883e-01]]

 [[ 7.14810193e-03 -3.30729410e-04]]

 [[-1.08355589e-01  2.85330087e-01]]

 [[-1.70078501e-01  4.40738946e-01]]

 [[-3.77093330e-02  1.16925925e-01]]

 [[-1.37459710e-01  3.84120107e-01]]

 [[-9.49004441e-02  2.20403567e-01]]

 [[-1.24720313e-01  3.63384038e-01]]

 [[-2.65789181e-01  7.02113032e-01]]

 [[-2.02227503e-01  5.91903210e-01]]

 [[-1.48623347e-01  4.01431829e-01]]

 [[-7.88750798e-02  1.87951535e-01]]

 [[-1.67266473e-01  4.84540254e-01]]

 [[-5.01479656e-02  1.31023169e-01]]

 [[-2.17148200e-01  5.70175707e-01]]

 [[-8.34355429e-02  2.44192317e-01]]

 [[-3.08132656e-02  1.01348393e-01]]

 [[-5.33434078e-02  1.35909379e-01]]

 [[-6.68721646e-02  2.12959155e-01]]

 [[-1.24343678e-01  3.13395441e-01]]

 [[-1.45675689e-01  4.49096173e-01]]

 [[-1.78463608e-01  5.55384099e-01]]

 [[-1.23088330e-01  3.09060633e-01]]

 [[-1.5769

IndexError: invalid index to scalar variable.